In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
main_data_df = pd.read_csv('./main_data_ver_2.3.csv')

Wall time: 10.9 s


In [3]:
main_data_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [4]:
main_data_df.drop(columns=['시군구', '번지', '본번', '부번', '도로명', '시', '구', '동'], inplace=True)
main_data_df.head()

,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,가격/면적,year,month,target_log_transformed,target/area_log_transformed,건물종류,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [5]:
main_data_df.head()

,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,가격/면적,year,month,target_log_transformed,target/area_log_transformed,건물종류,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [6]:
main_data_df.drop(columns=['대장구분코드'], inplace=True)

In [7]:
main_data_df.head()

,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,가격/면적,year,month,target_log_transformed,target/area_log_transformed,건물종류,trade_type,지번주소,대장구분명,도로명주소
0,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
1,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
2,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
3,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
4,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0


In [8]:
main_data_df.drop(columns=['trade_type'], inplace=True)

In [9]:
main_data_df.drop(columns=['계약년월'], inplace=True)
main_data_df.head()

,건물명,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,year,month,target_log_transformed,target/area_log_transformed,건물종류,지번주소,대장구분명,도로명주소
0,개포2차현대아파트(220),매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
1,개포2차현대아파트(220),매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
2,개포6차우성아파트1동~8동,매매,67.28,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
3,개포6차우성아파트1동~8동,매매,67.28,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
4,개포6차우성아파트1동~8동,매매,67.28,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0


In [10]:
main_data_df.rename(columns={'year':'년', 'month':'월'}, inplace=True)
main_data_df.head()

,건물명,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,대장구분명,도로명주소
0,개포2차현대아파트(220),매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
1,개포2차현대아파트(220),매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
2,개포6차우성아파트1동~8동,매매,67.28,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
3,개포6차우성아파트1동~8동,매매,67.28,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
4,개포6차우성아파트1동~8동,매매,67.28,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0


In [11]:
main_data_df['건물명'].nunique()

82411

In [12]:
main_data_df['건물명'].value_counts()[:100]

현대               26068
신동아              19229
두산               18327
한신               17989
주공2              17122
삼성래미안            16927
벽산               14664
파크리오             14249
우성               13799
삼성               13413
잠실엘스             12473
리센츠              12085
은마               11764
미성               10848
대우               10460
주공아파트 5단지         9181
극동                9153
주공5               8934
쌍용                8756
대림e-편한세상          8599
동아                8431
개포주공 1단지          8391
대림                8144
경남                7908
에스케이북한산시티         7695
트리지움              7655
삼익                7553
현대3               7465
현대1               7307
삼환                7137
선사현대아파트           7025
롯데캐슬              6806
올림픽훼밀리타운          6751
중계그린1단지           6673
우성1               6644
도곡렉슬              6557
중앙하이츠             6484
상계주공7(고층)         6476
남산타운              6405
성원                6288
주공3               6191
반포자이              6134
상계주공6(고층)         6021
삼익그린2차     

In [13]:
main_data_df['대장구분명'].value_counts()

일반       4553162
BL          3759
산           2091
가            508
지구BL         346
Name: 대장구분명, dtype: int64

In [14]:
ga_df = main_data_df[main_data_df['대장구분명'] == '가']
ga_df.head()

,건물명,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,대장구분명,도로명주소
340231,하버라인3단지,매매,74.8300,30,48600,0,16,2019,649.472137,2019,4,10.791379,6.476160,아파트,서울특별시 구로구 항동 가-242,가,서울특별시 구로구 항동로 60-0
340232,하버라인3단지,매매,84.8800,12,49000,0,6,2019,577.285580,2019,7,10.799576,6.358337,아파트,서울특별시 구로구 항동 가-242,가,서울특별시 구로구 항동로 43-0
340233,하버라인4단지,매매,59.8600,31,40000,0,12,2019,668.225860,2019,8,10.596635,6.504626,아파트,서울특별시 구로구 항동 가-238,가,서울특별시 구로구 항동로 72-0
340234,한양수자인에듀힐즈,매매,84.9544,29,49325,0,7,2019,580.605595,2019,10,10.806186,6.364072,아파트,서울특별시 구로구 항동 가-241,가,서울특별시 구로구 항동로 42-0
340235,한양수자인에듀힐즈,매매,84.9591,10,47400,0,2,2019,557.915515,2019,11,10.766378,6.324208,아파트,서울특별시 구로구 항동 가-241,가,서울특별시 구로구 항동로 42-0


In [15]:
jigubl_df = main_data_df[main_data_df['대장구분명'].str.contains('지구')]
print(jigubl_df.shape)
jigubl_df.head()

(346, 17)


,건물명,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,대장구분명,도로명주소
2071306,목동센트럴아이파크위브(1단지),전세,84.98,18,45000,0,6,2020,529.536361,2020,1,10.714418,6.272002,아파트,서울특별시 양천구 신월동 지구BL 1-0,지구BL,서울특별시 양천구 남부순환로83길 17-0
2071307,목동센트럴아이파크위브(1단지),전세,59.88,29,37000,0,6,2020,617.902472,2020,1,10.518673,6.426331,아파트,서울특별시 양천구 신월동 지구BL 1-0,지구BL,서울특별시 양천구 남부순환로83길 17-0
2071308,목동센트럴아이파크위브(1단지),전세,84.97,15,43000,0,5,2020,506.060963,2020,2,10.668955,6.226657,아파트,서울특별시 양천구 신월동 지구BL 1-0,지구BL,서울특별시 양천구 남부순환로83길 17-0
2071309,목동센트럴아이파크위브(1단지),전세,59.97,16,35500,0,11,2020,591.962648,2020,2,10.477288,6.383444,아파트,서울특별시 양천구 신월동 지구BL 1-0,지구BL,서울특별시 양천구 남부순환로83길 17-0
2071310,목동센트럴아이파크위브(1단지),전세,84.98,16,45000,0,9,2020,529.536361,2020,2,10.714418,6.272002,아파트,서울특별시 양천구 신월동 지구BL 1-0,지구BL,서울특별시 양천구 남부순환로83길 17-0


In [16]:
main_data_df.loc[jigubl_df.index, '대장구분명'] = '지구블록'
main_data_df['대장구분명'].value_counts()

일반      4553162
BL         3759
산          2091
가           508
지구블록        346
Name: 대장구분명, dtype: int64

In [17]:
main_data_df.head()

,건물명,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,대장구분명,도로명주소
0,개포2차현대아파트(220),매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
1,개포2차현대아파트(220),매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0
2,개포6차우성아파트1동~8동,매매,67.28,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
3,개포6차우성아파트1동~8동,매매,67.28,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0
4,개포6차우성아파트1동~8동,매매,67.28,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,서울특별시 강남구 개포동 658-1,일반,서울특별시 강남구 언주로 3-0


In [18]:
main_data_df.shape

(4559866, 17)

In [19]:
main_data_df['건물명'].value_counts()[:50]

현대           26068
신동아          19229
두산           18327
한신           17989
주공2          17122
삼성래미안        16927
벽산           14664
파크리오         14249
우성           13799
삼성           13413
잠실엘스         12473
리센츠          12085
은마           11764
미성           10848
대우           10460
주공아파트 5단지     9181
극동            9153
주공5           8934
쌍용            8756
대림e-편한세상      8599
동아            8431
개포주공 1단지      8391
대림            8144
경남            7908
에스케이북한산시티     7695
트리지움          7655
삼익            7553
현대3           7465
현대1           7307
삼환            7137
선사현대아파트       7025
롯데캐슬          6806
올림픽훼밀리타운      6751
중계그린1단지       6673
우성1           6644
도곡렉슬          6557
중앙하이츠         6484
상계주공7(고층)     6476
남산타운          6405
성원            6288
주공3           6191
반포자이          6134
상계주공6(고층)     6021
삼익그린2차        6021
롯데캐슬퍼스트       6010
상계주공9(고층)     5752
신동아아파트1       5747
목동신시가지14      5731
동부센트레빌        5557
한진한화그랑빌       5454
Name: 건물명, dtype: int64

In [25]:
addr_df = main_data_df[main_data_df['지번주소'] == '서울특별시 강서구 염창동 252-0']
addr_df

,건물명,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,대장구분명,도로명주소
3418139,현진파크뷰,전세,18.96,11,25000,0,8,2019,1318.565401,2019,12,10.126631,7.184300,다세대연립,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0
3903191,현진파크뷰,월세,18.96,11,3000,80,9,2019,158.227848,2019,2,8.006368,5.064036,다세대연립,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0
3903228,현진파크뷰,월세,18.96,10,3000,70,9,2019,158.227848,2020,3,8.006368,5.064036,다세대연립,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0
4157566,현진파크뷰,매매,28.41,16,25000,0,6,2019,879.971841,2019,3,10.126631,6.779890,오피스텔,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0
4157567,현진파크뷰,매매,28.41,17,25000,0,6,2019,879.971841,2019,3,10.126631,6.779890,오피스텔,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0
4157568,현진파크뷰,매매,28.41,3,25000,0,5,2019,879.971841,2019,4,10.126631,6.779890,오피스텔,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0
4157569,현진파크뷰,매매,28.41,7,25000,0,6,2019,879.971841,2019,4,10.126631,6.779890,오피스텔,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0
4157570,현진파크뷰,매매,28.41,1,25000,0,3,2019,879.971841,2019,5,10.126631,6.779890,오피스텔,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0
4157571,현진파크뷰,매매,28.41,17,25000,0,3,2019,879.971841,2019,5,10.126631,6.779890,오피스텔,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0
4157572,현진파크뷰,매매,28.41,11,25000,0,5,2019,879.971841,2019,6,10.126631,6.779890,오피스텔,서울특별시 강서구 염창동 252-0,일반,서울특별시 강서구 양천로75길 8-0


In [ ]:
main_data_df.drop(columns=['건물명'], inplace=True)

In [ ]:
main_data_df.head()

In [ ]:
main_data_df.to_csv('main_data_ver_2.5.csv', index=False)